In [39]:
import pandas as pd
import nltk
import json

# download nltk corpus (first time only)
# nltk.download("all")
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
# Installiere das SnowballStemmer für die deutsche Sprache
# nltk.download('stopwords')
# nltk.download('punkt')

## Helper functions

In [55]:
# the columns are beeing merged togheter here
def merge_columns(row):
    columns = [row.get('page_title', ''), row.get('sub_title', ''),
            row.get('introduction', ''), row.get('summary_box', ''),
            row.get('content', ''), row.get('accordion', '')]
    # columns = dataframe.columns.values.tolist()
    # li = [row.get(col, '') for col in columns if col != "url"]
    result = ' '.join(filter(None, columns)) #filter(None, li)
    return result

In [41]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [
        token for token in tokens if token not in stopwords.words("german")
    ]
    stemmer = SnowballStemmer("german")
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    processed_text = " ".join(stemmed_tokens)
    return processed_text

In [52]:
def get_sentiment(text_string):
    scores = analyzer.polarity_scores(text_string)
    return pd.Series(scores)

def add_sentiment_cols(dataframe, text_col="text"):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_cols = dataframe[text_col].apply(get_sentiment)
    dataframe = pd.concat([dataframe, sentiment_cols], axis=1)
    dataframe['judgement'] = dataframe['compound'].apply(lambda x: 'positive' if x > 0.05 else ("negative" if x < -0.05 else 'neutral'))
    return dataframe

In [113]:
def get_analysis_data(json_file):
    pandas_df = pd.read_json(json_file)
    filtered_df = pandas_df.loc[(pandas_df.loc[:, ~pandas_df.columns.isin(['url', 'page_title'])] != "").any(axis=1)] # exkludiert alle spalten von der Analyse die "" in den spalten ausser "page_title" haben 
    data = pd.DataFrame()
    data["text"] = filtered_df.apply(merge_columns, axis=1)
    data["text"] = data["text"].apply(preprocess_text)
    return add_sentiment_cols(data)

## Execution of Code

In [114]:
display(get_analysis_data("ratgeber_pages_part2.json"))

,text,neg,neu,pos,compound,judgement
1,selbstunfall auto – ? schuld ! sekund aufgepas...,0.025,0.975,0.000,-0.5627,negative
2,"invaliditat todesfall : sorg optimal vorsorg ,...",0.000,0.963,0.037,0.7820,positive
3,schad mietwohn schad haft mieterin miet ? freu...,0.039,0.961,0.000,-0.9749,negative
5,versich auto parkschad schad parkiert auto sol...,0.103,0.897,0.000,-0.9708,negative
6,online-shopping risiko schutz online-betrug in...,0.079,0.907,0.014,-0.9830,negative
...,...,...,...,...,...,...
143,auto bereit wint schnee eis sich auto unterweg...,0.009,0.974,0.017,0.3736,positive
144,tipps cyberkriminalitat leicht opf cyberkrimin...,0.021,0.968,0.011,-0.4329,negative
145,recht uberhol autobahn hoh kost fall unfall sc...,0.000,1.000,0.000,0.0000,neutral
146,rasenmahrobot : praktisch gartenhelf schutz op...,0.008,0.983,0.008,0.0000,neutral


#### Ich möchte url und page_title immer noch vor text